# RunableParallel

> LCEL가 병렬 요청을 지원하는 방법에 대해 정리함.

`langchain_core.runnables` 모듈의 `RunnableRarallel` 클래스를 활용해 두 가지 작업을 병렬로 처리하는 예시 학습

## 1. Set API Keys

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## 2. RunnableParallel

### Runnable 이란? 

> [공식 document](https://python.langchain.com/docs/concepts/runnables/)

Runnable 이란 LangChain에서 Prompt, LLMs, OutputParser, Retriever 등 각 컴포넌트를 구현한 방식을 의미함.

LangChain은 각 컴포넌트를 Runnable 프로토콜에 따라 구현함으로써,
- LCEL을 활용한 사용자 정의 체인을 쉽게 만들 수 있도록 함
- 모든 컴포넌트가 표준 인터페이스에 의해 동작하도록 구현함

우리는 LCEL을 활용해 Chain을 구성하려면 Runnable 프로토콜을 따르는 컴포넌트들을 사용해서 연결해야함
- 이렇게 생성한 Chain로 Runnable한 객체임.

### RunnableParallel 이란?

LECL을 활용해 생성한 복수의 Chain을 병렬로 동시에 수행할 수 있도록 해주는 클래스가 RuunableParallel 클래스임
- 각 체인에 대한 key를 설정해 파라미터로 입력해줌으로써 동시에 실행할 수 있는 객체를 생성함
- 이떄 지정하는 key 값의 의미는 : 해당 chain의 결과값을 지정한 key에 넣어준다는 의미임

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

chain_1 = (
    PromptTemplate.from_template("{country}의 수도는?")
    | ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
    | StrOutputParser()
)

chain_2 = (
    PromptTemplate.from_template("{country}의 면적은?")
    | ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
    | StrOutputParser()
)

In [4]:
from langchain_core.runnables import RunnableParallel

combined = RunnableParallel(capital=chain_1, area=chain_2)

In [5]:
combined.invoke({"country" : "미국"})

{'capital': '미국의 수도는 워싱턴 D.C.입니다.',
 'area': '미국의 면적은 약 9,830,000 평방킬로미터(3,796,000 평방마일)입니다. 이는 미국이 세계에서 세 번째로 큰 나라임을 의미합니다. 러시아와 캐나다에 이어 면적이 가장 큰 국가입니다.'}

### 2-1. 두 chain의 변수명이 다른 경우

> 단순히, 각 변수에 대한 값을 딕셔너리로 만들어서 전달! 내부에서 알아서 해당하는 변수에 값을 넣어 프롬프트를 완성시켜줌

In [7]:
chain_1 = (
    PromptTemplate.from_template("{country1}의 수도는?")
    | ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
    | StrOutputParser()
)

chain_2 = (
    PromptTemplate.from_template("{country2}의 면적은?")
    | ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
    | StrOutputParser()
)

combined = RunnableParallel(capital=chain_1, area=chain_2)

In [9]:
combined.invoke({"country1" : "미국", "country2" : "대한민국"})

{'capital': '미국의 수도는 워싱턴 D.C.입니다.',
 'area': '대한민국의 면적은 약 100,210 평방킬로미터(㎢)입니다. 이는 한반도의 남쪽 부분에 해당하며, 북한과의 경계를 포함한 면적입니다.'}

### 2-2. 배치 처리

In [11]:
combined.batch([
    {"country1" : "미국", "country2" : "대한민국"},
    {"country1" : "대한민국", "country2" : "일본"},
    {"country1" : "일본", "country2" : "중국"}
])

[{'capital': '미국의 수도는 워싱턴 D.C.입니다.',
  'area': '대한민국의 면적은 약 100,210 평방킬로미터입니다. 이는 한반도의 남쪽 부분에 해당하며, 북한과의 경계를 포함한 면적입니다.'},
 {'capital': '대한민국의 수도는 서울입니다.',
  'area': '일본의 면적은 약 377,975 평방킬로미터입니다. 이는 일본이 세계에서 62번째로 큰 나라임을 의미합니다.'},
 {'capital': '일본의 수도는 도쿄(東京)입니다.',
  'area': '중국의 면적은 약 9,596,961 평방킬로미터입니다. 이는 세계에서 세 번째로 큰 국가로, 러시아와 캐나다에 이어 위치하고 있습니다.'}]